In [56]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np
from torch.utils.data import TensorDataset

In [61]:
#read in file
df = pd.read_parquet("small_tokenized_data.parquet")

In [63]:
#seperate review ids and append to newIDS list
newIDS = df["input_ids"].tolist()

In [64]:
#Identify attention mask part of the column and create a list attn_mask
attn_mask = df["attention_mask"].tolist()

In [65]:
#create a list containing star column information
starIDS = df["stars"].tolist()

In [66]:
#Create a pretrained bert model
bert = BertModel.from_pretrained('bert-base-uncased')

In [67]:
#load data in batches for bert
bertData = TensorDataset(torch.tensor(newIDS), torch.tensor(attn_mask), torch.tensor(starIDS))
# set up a data loader to get batches from our dataset
bertLoader = torch.utils.data.DataLoader(bertData, batch_size = 16, shuffle = False)

In [55]:
#Create individual files for the vectors and labels 
i = 0
for batch in bertLoader:
    y, pooled = bert(batch[0])
    torch.save(pooled, ("vector"+"%d"+".txt")%i)
        
    torch.save(batch[2], ("label"+"%d"+".txt")%i)

    print(i)
    i = i+1

'\ni = 0\nfor batch in bertLoader:\n    y, pooled = bert(batch[0])\n    torch.save(pooled, ("vector"+"%d"+".txt")%i)\n        \n    torch.save(batch[2], ("label"+"%d"+".txt")%i)\n\n    print(i)\n    i = i+1\n'